In [1]:
import asyncio
import base64
import os
from io import BytesIO

import httpx
import orjson
import polars as pl
from dotenv import load_dotenv
from PIL import Image

load_dotenv()

True

In [2]:
assert os.getenv("OPENROUTER_API_KEY"), "OpenRouter API key is not defined in .env."
model = "google/gemini-2.5-flash-lite-preview-06-17"


API_URL = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
    "Content-Type": "application/json",
}

In [3]:
system = "Respond in only one (1) paragraph."
prompt = "What is love?"

params = {
    "model": model,
    # "model": "openai/gpt-4o",
    "messages": [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ],
}

r = httpx.post(url=API_URL, headers=headers, data=orjson.dumps(params))

In [4]:
print(orjson.dumps(r.json(), option=orjson.OPT_INDENT_2).decode("utf-8"))

{
  "id": "gen-1752963521-01FnLY7trOckZhSXNY77",
  "provider": "Google",
  "model": "google/gemini-2.5-flash-lite-preview-06-17",
  "object": "chat.completion",
  "created": 1752963521,
  "choices": [
    {
      "logprobs": null,
      "finish_reason": "stop",
      "native_finish_reason": "STOP",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Love is a complex and multifaceted emotion characterized by deep affection, care, and connection towards another person, often involving a sense of intimacy, commitment, and desire for their well-being and happiness. It can manifest in various forms, from the passionate romance between partners to the unconditional bond of family, the steadfast loyalty of friendship, and even a broader empathy for humanity, all united by a powerful emotional investment and a willingness to prioritize the beloved's needs and happiness.",
        "refusal": null,
        "reasoning": null
      }
    }
  ],
  "usage": {
    "

Test image inputs using local b64.


In [5]:
img = Image.open("profpic.webp")
# img.resize((256, 256))

In [6]:
buffered = BytesIO()
img.resize((768, 768)).save(buffered, format="WEBP")
img_base64 = base64.b64encode(buffered.getvalue())
img_base64_str = img_base64.decode("utf-8")

img_base64_str[0:100]

'UklGRtquAQBXRUJQVlA4IM6uAQCQzwWdASoAAwADPm0ukkYkIqGkrNOc2JANiU2g80KZLIGyOQCJZH0YcEVbrm77N0D7Lus5i33v'

In [7]:
system = "Describe the image the user provides in no more than two (2) sentences."

params = {
    "model": model,
    # "model": "openai/gpt-4o",
    "messages": [
        {"role": "system", "content": system},
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/webp;base64,{img_base64_str}"},
                }
            ],
        },
    ],
}

r = httpx.post(url=API_URL, headers=headers, data=orjson.dumps(params))

In [8]:
print(orjson.dumps(r.json(), option=orjson.OPT_INDENT_2).decode("utf-8"))

{
  "id": "gen-1752963523-SpSD8cSfmE2xxnztvYUL",
  "provider": "Google AI Studio",
  "model": "google/gemini-2.5-flash-lite-preview-06-17",
  "object": "chat.completion",
  "created": 1752963523,
  "choices": [
    {
      "logprobs": null,
      "finish_reason": "stop",
      "native_finish_reason": "STOP",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "A young man with brown hair is smiling and looking at the camera. He is wearing a gray fleece jacket over a blue t-shirt. The background features a tree with vibrant orange leaves, suggesting it is autumn.",
        "refusal": null,
        "reasoning": null
      }
    }
  ],
  "usage": {
    "prompt_tokens": 276,
    "completion_tokens": 44,
    "total_tokens": 320
  }
}


## Use async to Parallelize Calls to Different Models.

Define as a list of key-values where the key is the humanized form of the model for better reporting.


In [9]:
model_list = [
    {"Gemini 2.5 Flash Lite": "google/gemini-2.5-flash-lite-preview-06-17"},
    {"Claude Sonnet 4": "anthropic/claude-sonnet-4"},
    {"GPT-4.1 Mini": "openai/gpt-4.1-mini"},
]

In [10]:
async def query_image_async(model_kv, client, system, img_base64_str):
    model_name, model_openrouter = list(model_kv.items())[0]
    params = {
        "model": model_openrouter,
        "messages": [
            {"role": "system", "content": system},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/webp;base64,{img_base64_str}"
                        },
                    }
                ],
            },
        ],
    }

    r = await client.post(url=API_URL, headers=headers, data=orjson.dumps(params))
    return {
        "model": model_name,
        "response": r.json()["choices"][0]["message"]["content"],
    }

In [11]:
client = httpx.AsyncClient()
await query_image_async(model_list[0], client, system, img_base64_str)

{'model': 'Gemini 2.5 Flash Lite',
 'response': 'A smiling man is captured in a close-up portrait, set against a vibrant backdrop of autumn foliage. He is wearing a gray fleece jacket over a blue shirt.'}

Query the models simultaneously.

NB: `asyncio.gather()` will preserve order of inputs: https://docs.python.org/3/library/asyncio-task.html#running-tasks-concurrently


In [12]:
async def query_models_async(model_list, client, system, img_base64_str):
    queries = [
        query_image_async(model, client, system, img_base64_str) for model in model_list
    ]

    results = await asyncio.gather(*queries)
    return results

In [13]:
results = await query_models_async(model_list, client, system, img_base64_str)
results

[{'model': 'Gemini 2.5 Flash Lite',
  'response': 'A young man smiles at the camera, with bright orange and red autumn leaves behind him. He is wearing a grey fleece jacket over a blue shirt.'},
 {'model': 'Claude Sonnet 4',
  'response': "The image shows a smiling young man with short brown hair wearing a gray North Face jacket over a blue shirt. He's positioned in front of vibrant autumn foliage displaying brilliant orange and red leaves, creating a beautiful fall backdrop."},
 {'model': 'GPT-4.1 Mini',
  'response': 'The image shows a smiling man wearing a gray fleece jacket and a blue shirt, standing outdoors in front of a tree with vibrant orange leaves. The background also includes a chain-link fence.'}]

In [14]:
with pl.Config() as cfg:
    cfg.set_tbl_formatting("ASCII_MARKDOWN")
    cfg.set_fmt_str_lengths(10**5)
    cfg.set_tbl_width_chars(-1)
    cfg.set_tbl_hide_column_data_types(True)
    cfg.set_tbl_hide_dataframe_shape(True)

    print(pl.from_dicts(results))

| model                 | response                                                                                                                                                                                                                                         |
|-----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Gemini 2.5 Flash Lite | A young man smiles at the camera, with bright orange and red autumn leaves behind him. He is wearing a grey fleece jacket over a blue shirt.                                                                                                     |
| Claude Sonnet 4       | The image shows a smiling young man with short brown hair wearing a gray North Face jacket over a blue shirt. He's positioned in front of vibrant autumn foliage displa